In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re
import string
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
# import torchutils as tu
from torchmetrics.classification import BinaryAccuracy
import nltk
from nltk.corpus import stopwords
import pymorphy3

from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline

In [2]:
path = "/home/savely/ds_bootcamp/ds-phase-2/Natural-Language-Processing/data/healthcare_facilities_reviews.jsonl"

In [3]:
df = pd.read_json(path, lines=True)

In [4]:
df.head()

,review_id,category,title,content,sentiment,source_url
0,0,Поликлиники стоматологические,Классный мастер,Огромное спасибо за чудесное удаление двух зуб...,positive,http://www.spr.ru/forum_vyvod.php?id_tema=2727539
1,1,Поликлиники стоматологические,Замечательный врач,Хочу выразить особую благодарность замечательн...,positive,http://www.spr.ru/forum_vyvod.php?id_tema=2302877
2,2,Поликлиники стоматологические,Благодарность работникам рентгена,Добрый вечер! Хотелось бы поблагодарить сотруд...,positive,http://www.spr.ru/forum_vyvod.php?id_tema=2815031
3,3,Поликлиники стоматологические,Доктор Рабинович,Женщины советского образца в регистратуре не и...,negative,http://www.spr.ru/forum_vyvod.php?id_tema=3443161
4,4,Поликлиники стоматологические,Есть кому сказать спасибо,У меня с детства очень плохие зубы (тонкая и х...,positive,http://www.spr.ru/forum_vyvod.php?id_tema=2592430


In [5]:
df["sentiment"].value_counts()

sentiment
positive    41419
negative    29178
Name: count, dtype: int64

In [6]:
# Подготовка данных для обучения
x = df['content']  # Тексты отзывов (фичи)
y = df['sentiment']   # Целевая переменная (метки классов)

In [7]:
# Загрузка стоп-слов
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/savely/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Инициализируем лемматизатор
morph = pymorphy3.MorphAnalyzer()

In [9]:
# Инициализируем список стоп-слов
stop_words = set(stopwords.words('russian'))

# Функция для очистки текста
def clean_text(text):
    if not isinstance(text, str):  # Проверяем, что текст строка
        return ""

    text = text.lower()  # Приводим к нижнему регистру
    text = text.translate(str.maketrans('', '', string.punctuation))  # Удаляем пунктуацию
    tokens = text.split()  # Разбиваем на слова
    tokens = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]  # Лемматизация
    tokens = [word for word in tokens if len(word) > 1]  # Убираем слишком короткие слова
    cleaned_text = " ".join(tokens).strip()  # Убираем лишние пробелы
    return cleaned_text if cleaned_text else None 


In [10]:
x[0]

'Огромное спасибо за чудесное удаление двух зубов мудрости за мгновение, доктор Матвеев! Профессионал с большой буквы. Боялась страшно, но все заняло реально 10 секунд, и я согласилась на удаление сразу и второго зуба! Без боли и страха. Очень рекомендую.'

In [11]:
clean_text(x[0])

'огромный спасибо чудесный удаление два зуб мудрость мгновение доктор матвеев профессионал большой буква бояться страшно занять реально 10 секунда согласиться удаление сразу второй зуб боль страх очень рекомендовать'

In [12]:
# Предобработка текста
X_processed = x.apply(clean_text)

In [13]:
# Кодирование целевой переменной
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [14]:
# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_encoded, test_size=0.2, random_state=42)

In [15]:
# Вариант 1: Классификация с Bag-of-Words + Logistic Regression
bow_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=10000, ngram_range=(1,2))),
    ('classifier', LogisticRegression(max_iter=500))
])

bow_pipeline.fit(X_train, y_train)
bow_pred = bow_pipeline.predict(X_test)

print("Bag-of-Words + Logistic Regression:")
print(classification_report(y_test, bow_pred, target_names=encoder.classes_))

Bag-of-Words + Logistic Regression:
              precision    recall  f1-score   support

    negative       0.92      0.92      0.92      5778
    positive       0.95      0.95      0.95      8342

    accuracy                           0.94     14120
   macro avg       0.93      0.93      0.93     14120
weighted avg       0.94      0.94      0.94     14120



In [16]:
# Вариант 2: Pipeline с TF-IDF + Logistic Regression
print("\nОбучение модели TF-IDF + Logistic Regression...")
tfidf_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=10000, ngram_range=(1, 2))),
    ('classifier', LogisticRegression(max_iter=500))
])

tfidf_pipeline.fit(X_train, y_train)
tfidf_pred = tfidf_pipeline.predict(X_test)

print("\nРезультаты TF-IDF:")
print(classification_report(y_test, tfidf_pred, target_names=encoder.classes_))


Обучение модели TF-IDF + Logistic Regression...

Результаты TF-IDF:
              precision    recall  f1-score   support

    negative       0.93      0.94      0.93      5778
    positive       0.95      0.95      0.95      8342

    accuracy                           0.94     14120
   macro avg       0.94      0.94      0.94     14120
weighted avg       0.94      0.94      0.94     14120



In [17]:
# Пример предсказания для нового текста
test_review = "Очень плохой сервис, никогда больше не приду!"
processed_review = clean_text(test_review)
bow_prediction = bow_pipeline.predict([processed_review])[0]
tfidf_prediction = tfidf_pipeline.predict([processed_review])[0]

print(f"\nПример работы модели для текста: '{test_review}'")
print(f"Bag-of-Words предсказывает: {encoder.inverse_transform([bow_prediction])[0]}")
print(f"TF-IDF предсказывает: {encoder.inverse_transform([tfidf_prediction])[0]}")



Пример работы модели для текста: 'Очень плохой сервис, никогда больше не приду!'
Bag-of-Words предсказывает: negative
TF-IDF предсказывает: negative


In [18]:
import joblib
# Сохраняем pipeline (модель + векторизатор)
joblib.dump(bow_pipeline, 'bow_pipeline.joblib')
joblib.dump(tfidf_pipeline, 'tfidf_pipeline.joblib')


['tfidf_pipeline.joblib']